# Automated ML

Importing all needed dependencies related to the project

In [2]:
import logging
import os
import json
import csv
import numpy as np
import pandas as pd
import pkg_resources
import joblib

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.pipeline.steps import AutoMLStep
from azureml.widgets import RunDetails
from azureml.core import Model, Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
from sklearn.preprocessing import StandardScaler

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.27.0


In [3]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

In [8]:
# azureml-core of version 1.0.72 or higher is required
# azureml-dataprep[pandas] of version 1.1.34 or higher is required
from azureml.core import Workspace, Dataset

subscription_id = '81cefad3-d2c9-4f77-a466-99a7f541c7bb'
resource_group = 'aml-quickstarts-145303'
workspace_name = 'quick-starts-ws-145303'

workspace = Workspace(subscription_id, resource_group, workspace_name)

dataset = Dataset.get_by_name(workspace, name='diabetes-dataset')
df = dataset.to_pandas_dataframe()

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
Pregnancies                 768 non-null int64
Glucose                     768 non-null int64
BloodPressure               768 non-null int64
SkinThickness               768 non-null int64
Insulin                     768 non-null int64
BMI                         768 non-null float64
DiabetesPedigreeFunction    768 non-null float64
Age                         768 non-null int64
Outcome                     768 non-null int64
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [10]:
#Display the first five records of the dataset
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [12]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for the cluster
cpu_cluster_name = "ComputeCluster"

# Verifing that cluster does not exist already or Create a New Cluster
try:
    compute_target = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster.')
except ComputeTargetException:
    print('Creating a new compute cluster....')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

# Can poll for a minimum number of nodes and for a specific timeout. 
# If no min node count is provided it uses the scale settings for the cluster.
compute_target.wait_for_completion(show_output=True)

# get_status() to get a detailed status for the current running cluster 
print(compute_target.get_status().serialize())

Found existing cluster.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 2, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-05-21T06:29:01.053000+00:00', 'errors': None, 'creationTime': '2021-05-21T06:26:56.693359+00:00', 'modifiedTime': '2021-05-21T06:27:12.055678+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 2, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}


# Compute Cluster configuration for this project:

{'currentNodeCount': 2, 'targetNodeCount': 2, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 2, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Steady', 'allocationStateTransitionTime': '2021-05-21T06:29:01.053000+00:00', 'errors': None, 'creationTime': '2021-05-21T06:26:56.693359+00:00', 'modifiedTime': '2021-05-21T06:27:12.055678+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 2, 'maxNodeCount': 5, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_DS3_V2'}

## AutoML Configuration

Automl settings and configuration used for this peoject:


In [13]:
# Automl settings 
automl_settings = {
    "experiment_timeout_minutes": 50,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy',
    "n_cross_validations": 5,
    "iterations": 30
    
}

# Automl configuration
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = 'classification',
                             training_data=dataset,
                             label_column_name='Outcome',
                             enable_early_stopping= True,
                             featurization = 'auto',
                             debug_log = 'automl_errors.log',
                             **automl_settings
                            )



In [14]:
# Submit experiment with AutoML Configuration

remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on ComputeCluster with default configuration
Running on remote compute: ComputeCluster


Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_81963d46-87b4-4961-a3e7-ae90b262a8ca,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

******************************************************************

## Run Details

Now we will see best models for this classification problems.We can use the`RunDetails` widget to show various experiments performed.You can also see best model in ML Azure Portal.

In [17]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [18]:
remote_run.wait_for_completion(show_output=True)

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_81963d46-87b4-4961-a3e7-ae90b262a8ca,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

****************************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

****************************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more abo

{'runId': 'AutoML_81963d46-87b4-4961-a3e7-ae90b262a8ca',
 'target': 'ComputeCluster',
 'status': 'Completed',
 'startTimeUtc': '2021-05-21T06:36:53.07491Z',
 'endTimeUtc': '2021-05-21T06:50:32.844501Z',
 'properties': {'num_iterations': '30',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '5',
  'target': 'ComputeCluster',
  'AMLSettingsJsonString': '{"path":null,"name":"automl","subscription_id":"81cefad3-d2c9-4f77-a466-99a7f541c7bb","resource_group":"aml-quickstarts-145303","workspace_name":"quick-starts-ws-145303","region":"southcentralus","compute_target":"ComputeCluster","spark_service":null,"azure_service":"remote","many_models":false,"pipeline_fetch_max_batch_size":1,"enable_batch_run":false,"enable_run_restructure":false,"start_auxiliary_runs_before_parent_complete":false,"iterations":30,"primary_metric":"accuracy","task_type":"classification","data_s

In [19]:
remote_run

Experiment,Id,Type,Status,Details Page,Docs Page
automl,AutoML_81963d46-87b4-4961-a3e7-ae90b262a8ca,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation


## Best Model

Now we will see best model from the azure automl experiments.



In [20]:
# Retrieve and save best automl model
best_model, model = remote_run.get_output()
print(best_model)
print(model.steps)

Run(Experiment: automl,
Id: AutoML_81963d46-87b4-4961-a3e7-ae90b262a8ca_28,
Type: azureml.scriptrun,
Status: Completed)
[('datatransformer', DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                feature_sweeping_config=None, feature_sweeping_timeout=None,
                featurization_config=None, force_text_dnn=None,
                is_cross_validation=None, is_onnx_compatible=None, logger=None,
                observer=None, task=None, working_dir=None)), ('prefittedsoftvotingclassifier', PreFittedSoftVotingClassifier(classification_labels=None,
                              estimators=[('22',
                                           Pipeline(memory=None,
                                                    steps=[('maxabsscaler',
                                                            MaxAbsScaler(copy=True)),
                                                           ('lightgbmclassifier',
                                                            Lig

In [21]:
print(model)

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('prefittedsoftvotingclassifier',...
                                                                                                penalty='l2',
                                                                                                random_state=None,
                                                                                                solver='lbfgs',
                                                       

In [22]:
print(best_model)

Run(Experiment: automl,
Id: AutoML_81963d46-87b4-4961-a3e7-ae90b262a8ca_28,
Type: azureml.scriptrun,
Status: Completed)


In [23]:
remote_run.get_metrics()

{'experiment_status': ['DatasetEvaluation',
  'FeaturesGeneration',
  'DatasetFeaturization',
  'DatasetFeaturizationCompleted',
  'DatasetCrossValidationSplit',
  'ModelSelection',
  'BestRunExplainModel',
  'ModelExplanationDataSetSetup',
  'PickSurrogateModel',
  'EngineeredFeatureExplanations',
  'EngineeredFeatureExplanations',
  'RawFeaturesExplanations',
  'RawFeaturesExplanations',
  'BestRunExplainModel'],
 'experiment_status_description': ['Gathering dataset statistics.',
  'Generating features for the dataset.',
  'Beginning to fit featurizers and featurize the dataset.',
  'Completed fit featurizers and featurizing the dataset.',
  'Generating individually featurized CV splits.',
  'Beginning model selection.',
  'Best run model explanations started',
  'Model explanations data setup completed',
  'Choosing LightGBM as the surrogate model for explanations',
  'Computation of engineered features started',
  'Computation of engineered features completed',
  'Computation of ra

## Model Deployment

We have to deploy only one of the two models trained.. We will Perform the steps in the rest of this notebook only if we wish to deploy this model.

Now we have registered the model, created an inference config and deployed the model as a web service for endpoints consumption using professional API Testing tools (Swagger UI).

In [24]:
# Registring the best model
model = best_model.register_model(model_name='automl-diabetes-best-model',model_path='outputs/model.pkl')


In [25]:
# Now we will get environment with dependencies
environment = Environment.get(ws, "AzureML-AutoML")

In [27]:
inference_config = InferenceConfig(entry_script='scoring.py',
                                   environment=environment)
service_name = 'automl-deploy'
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

service = Model.deploy(workspace=ws,
                       name=service_name,
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config,
                       overwrite=True
                      )
service.wait_for_deployment(show_output=True)

scoring_uri = service.scoring_uri
print(scoring_uri)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-05-21 07:12:29+00:00 Creating Container Registry if not exists..
2021-05-21 07:12:40+00:00 Registering the environment..
2021-05-21 07:12:41+00:00 Use the existing image.
2021-05-21 07:12:41+00:00 Generating deployment configuration.
2021-05-21 07:12:42+00:00 Submitting deployment to compute.
2021-05-21 07:12:46+00:00 Checking the status of deployment automl-deploy..
2021-05-21 07:14:09+00:00 Checking the status of deployment automl-deploy..
2021-05-21 07:17:12+00:00 Checking the status of inference endpoint automl-deploy.
Succeeded
ACI service creation operation finished, operation "Succeeded"
http://a2d23cdc-634e-42e8-a1d7-770bcba56b69.southcentralus.azurecontainer.io/score


In [28]:
# Now we will try to Enable app insights
service.update(enable_app_insights=True)

Now we will consume the deployed webservice in AutoML so we are sending the web service requests to deployed best model

In [29]:
import requests
import json

#two set of data to score, so we get two results back
data = {"data": [{"Pregnancies": 9, 
     "Glucose": 130, 
     "BloodPressure": 60, 
     "SkinThickness": 20, 
     "Insulin": 6, 
     "BMI": 40.5, 
     "DiabetesPedigreeFunction": 0.537, 
     "Age": 50},

    {"Pregnancies": 8, 
     "Glucose": 70, 
     "BloodPressure": 80, 
     "SkinThickness": 30, 
     "Insulin": 18, 
     "BMI": 24.5, 
     "DiabetesPedigreeFunction": 0.951, 
     "Age": 38},
      ]}
    
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
#headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())
print("Case 0 : Not Diabetic, Case 1 : Diabetic.")

{"result": [1, 0]}
Case 0 : Not Diabetic, Case 1 : Diabetic.


Print the logs of the web service and delete the service

In [31]:
logs = service.get_logs()
logs

'2021-05-21T07:22:54,479063800+00:00 - gunicorn/run \n2021-05-21T07:22:54,487931800+00:00 - rsyslog/run \n2021-05-21T07:22:54,498133500+00:00 - nginx/run \n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n/usr/sbin/nginx: /azureml-envs/azureml_600883adcde8eb30a3a0fa1bdca2e09e/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)\n2021-05-21T07:22:54,519364400+00:00 - iot-server/run 

In [32]:
#Deleting the deployed web service

service.delete()